In [1]:
import axisrules as axis
import numpy as np
import pandas as pd
import math
import time

In [2]:
# load files with pandas 
df_spo = pd.read_csv('data/approval_assembly_spo.csv')
parties = list(df_spo.columns)
votes_spo = df_spo.to_numpy()
votes_sps = pd.read_csv('data/approval_assembly_sps.csv').to_numpy()
votes_spo = np.concatenate([votes_spo, votes_sps])

In [3]:
votes_spo = np.maximum(votes_spo, 0)
votes_sps = np.maximum(votes_sps, 0)
print(len(votes_sps))
print(len(votes_spo))

23
2500


In [4]:
axis_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, axis.MinimalSwaps, axis.ForbiddenTriples]
axis_rules_seq = [axis.SequentialVoterDeletion, axis.SequentialMinimalFlips, 
              axis.SequentialBallotCompletion, axis.SequentialMinimalSwaps, axis.SequentialForbiddenTriples]

for i,rule in enumerate(axis_rules):
    print(rule.name)
    t1 = time.time()
    seq_axis = axis_rules_seq[i](votes_spo)()
    rule_instance = rule(votes_spo)
    res = rule_instance.bruteforce(proxy_axis=seq_axis)
    axis_score = rule_instance.get_score(res[0][0])
    axis.print_axis( res[0][0], parties)
    print("score :", axis_score)
    t2 = time.time()
    print("Time :", t2-t1)

Voter Deletion
RE < Dem < HOR < LR < RN < SOC < Écolo < LFI < GDR < LIOT
score : 910.0
Time : 1.8345158100128174
Minimal Flips
RE < Dem < HOR < LR < RN < SOC < Écolo < LFI < GDR < LIOT
score : 1206.0
Time : 6.896348714828491
Ballot Completion
GDR < Écolo < LFI < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 1759.0
Time : 4.256380796432495
Minimal Swaps
GDR < LFI < Écolo < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 2770.0
Time : 7.72849702835083
Forbidden Triples
GDR < LFI < Écolo < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 10809.0
Time : 34.053823709487915


In [5]:
axis_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, axis.MinimalSwaps, axis.ForbiddenTriples]

for rule in axis_rules:
    print(rule.name)
    t1 = time.time()
    rule_instance = rule(votes_spo)
    res = rule_instance.bruteforce()
    axis_score = rule_instance.get_score(res[0][0])
    axis.print_axis( res[0][0], parties)
    print("score :", axis_score)
    t2 = time.time()
    print("Time :", t2-t1)

Voter Deletion
RE < Dem < HOR < LR < RN < SOC < Écolo < LFI < GDR < LIOT
score : 910.0
Time : 2.1336326599121094
Minimal Flips
RE < Dem < HOR < LR < RN < SOC < Écolo < LFI < GDR < LIOT
score : 1206.0
Time : 8.58723521232605
Ballot Completion
GDR < Écolo < LFI < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 1759.0
Time : 4.272770643234253
Minimal Swaps
GDR < LFI < Écolo < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 2770.0
Time : 11.667797803878784
Forbidden Triples
GDR < LFI < Écolo < SOC < LIOT < RN < LR < Dem < HOR < RE
score : 10809.0
Time : 34.874653577804565


In [4]:
axis_rules = [axis.VoterDeletion, axis.MinimalFlips, axis.BallotCompletion, axis.MinimalSwaps, axis.ForbiddenTriples]

axis_rules_seq = [axis.SequentialVoterDeletion, axis.SequentialMinimalFlips, 
              axis.SequentialBallotCompletion, axis.SequentialMinimalSwaps, axis.SequentialForbiddenTriples]

for i, rule in enumerate(axis_rules_seq):
    print(rule.name)
    t1 = time.time()
    rule_instance = rule(votes_spo)
    res = rule_instance()
    axis_score = axis_rules[i](votes_spo).get_score(res)
    axis.print_axis( res, parties)
    print("score :", axis_score)
    t2 = time.time()
    print("Time :", t2-t1)

Sequential Voter Deletion
RN < LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT
score : 1188.0
Time : 0.08130645751953125
Sequential Minimal Flips
RN < LR < HOR < RE < Dem < LIOT < SOC < Écolo < LFI < GDR
score : 1564.0
Time : 0.13173818588256836
Sequential Ballot Completion
RN < LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT
score : 4021.0
Time : 0.09052896499633789
Sequential Minimal Swaps
RE < Dem < HOR < LR < RN < LIOT < GDR < Écolo < LFI < SOC
score : 3276.0
Time : 0.12403535842895508
Sequential Forbidden Triples
LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT < RN
score : 18806.0
Time : 0.12259221076965332


In [20]:
def matrix_coapproval(votes):
    n = len(votes)
    m = len(votes[0])
    coapproval = np.zeros((m,m))
    for i in range(m):
        for j in range(i+1,m):
            for k in range(n):
                if votes[k][i] ==  votes[k][j]:
                    coapproval[i][j] += 1
                    coapproval[j][i] += 1
    return coapproval/len(votes)


def is_valid(partial_axis, vote):
    started = False
    finished = False
    count = 0
    total = np.sum(vote)
    for i in range(len(partial_axis)):
        if vote[partial_axis[i]] == 1:
            count += 1
            if not started:
                started = True
            if finished:
                return False
        if vote[partial_axis[i]] == 0 and started:
            finished = True

    if total != count and count > 0 and (vote[partial_axis[0]] == 0 and vote[partial_axis[-1]] == 0):
        return False
        
    return True

def vote2ballot(vote):
    return [parties[x] for x in range(len(vote)) if vote[x] == 1]


def GreedyAxis(votes):
    matrix_coapp = matrix_coapproval(votes)
    v = matrix_coapp.ravel().argmax()
    v_x, v_y = np.unravel_index(v, matrix_coapp.shape)
    axis_out = [v_x, v_y]
    n, m = votes.shape
    current_vote_list = votes.copy()
    for k in range(m-2):
        matrix_coapp = matrix_coapproval(current_vote_list)
        left_coapp = matrix_coapp[axis_out[0], :]
        right_coapp = matrix_coapp[axis_out[-1], :]
        for x in axis_out:
            left_coapp[x] = 0
            right_coapp[x] = 0
        argmax_left = np.argmax(left_coapp)
        argmax_right = np.argmax(right_coapp)
        if matrix_coapp[axis_out[0], argmax_left] > matrix_coapp[axis_out[-1], argmax_right]:
            axis_out = [argmax_left] + axis_out
        else:
            axis_out = axis_out + [argmax_right]
        current_vote_list = [vote for vote in current_vote_list if is_valid(axis_out, vote)]

        # print(k, len(current_vote_list), [parties[x] for x in axis_out])

    return axis_out

def GreedyVoterDeletion(votes):
    matrix_coapp = matrix_coapproval(votes)
    v = matrix_coapp.ravel().argmax()
    v_x, v_y = np.unravel_index(v, matrix_coapp.shape)
    axis_out = [v_x, v_y]
    n, m = votes.shape
    current_vote_list = votes.copy()
    for k in range(m-2):
        on_left = np.zeros(m)
        on_right = np.zeros(m)
        for i in range(m):
            if i not in axis_out:
                
                tested_axis = [i] + axis_out
                on_left[i] = len([vote for vote in current_vote_list if is_valid(tested_axis, vote)])
                tested_axis = axis_out + [i]
                on_right[i] = len([vote for vote in current_vote_list if is_valid(tested_axis, vote)])

        # print(on_left, on_right)
        if np.max(on_left) > np.max(on_right):
            v = np.argmax(on_left)
            axis_out = [v] + axis_out
        else:
            v = np.argmax(on_right)
            axis_out = axis_out + [v]
        current_vote_list = [vote for vote in current_vote_list if is_valid(axis_out, vote)]
        # print(k, len(current_vote_list), [parties[x] for x in axis_out])
    return axis_out


print("Greedy Axis")
t1 = time.time()
res = GreedyAxis(votes_spo)
axis.print_axis(res, parties)
t2 = time.time()
print("Time :", t2-t1)

print("Greedy Voter Deletion")
t1 = time.time()
res = GreedyVoterDeletion(votes_spo)
axis.print_axis(res, parties)
t2 = time.time()
print("Time :", t2-t1)

Greedy Axis
RN < LR < HOR < RE < Dem < LIOT < GDR < Écolo < LFI < SOC
Time : 0.46065378189086914
Greedy Voter Deletion
RN < LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT
Time : 0.7640466690063477


In [28]:
def minBallotCompletionScore(partial_axis, vote):
    last = 0
    tab = [0]
    for i in range(len(partial_axis)):
        if vote[partial_axis[i]] == last:
            tab[-1] += 1
        else:
            tab.append(1)
            last = vote[partial_axis[i]]

    count = sum([tab[i] for i in range(len(tab)) if i%2 == 1])
    total = sum(vote)
    bad = 0
    for i in range(2,len(tab)):
        if i%2 == 0 and i < len(tab)-1:
            bad += tab[i]

    if tab[0] > 0 and len(tab)%2 == 1 and count > 0 and count != total:
        bad += min(tab[0], tab[-1])
        
    return bad


def GreedyBallotCompletion(votes):
    matrix_coapp = matrix_coapproval(votes)
    v = matrix_coapp.ravel().argmax()
    v_x, v_y = np.unravel_index(v, matrix_coapp.shape)
    axis_out = [v_x, v_y]
    n, m = votes.shape
    for k in range(m-2):
        on_left = np.ones(m)*m*n
        on_right = np.ones(m)*m*n
        for i in range(m):
            if i not in axis_out:
                
                tested_axis = [i] + axis_out
                on_left[i] = sum([minBallotCompletionScore(tested_axis, vote) for vote in votes])
                tested_axis = axis_out + [i]
                on_right[i] = sum([minBallotCompletionScore(tested_axis, vote) for vote in votes])

        # print(on_left, on_right)
        # print(k, [parties[x] for x in axis_out])
        if np.min(on_left) < np.min(on_right):
            v = np.argmin(on_left)
            axis_out = [v] + axis_out
        else:
            v = np.argmin(on_right)
            axis_out = axis_out + [v]
    return axis_out

print("Greedy Ballot Completion")
t1 = time.time()
res = GreedyBallotCompletion(votes_spo)
axis.print_axis(res, parties)
t2 = time.time()
print("Time :", t2-t1)


Greedy Ballot Completion
RN < LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT
Time : 0.6437878608703613


In [31]:
def minForbiddenTriplesScore(partial_axis, vote):
    last = 0
    tab = [0]
    for i in range(len(partial_axis)):
        if vote[partial_axis[i]] == last:
            tab[-1] += 1
        else:
            tab.append(1)
            last = vote[partial_axis[i]]

    count = sum([tab[i] for i in range(len(tab)) if i%2 == 1])
    total = sum(vote)
    score_from_inside = 0
    left = 0
    right = count
    for i in range(1,len(tab)):
        if i%2 == 1:
            left += tab[i]
            right -= tab[i]
        else:
            score_from_inside += left*right*tab[i]

    score_from_left = 0
    remaining = total - count
    right = count
    for i in range(len(tab)):
        
        if i%2 == 1:
            right -= tab[i]
        else:
            score_from_left += remaining*right*tab[i]

    score_from_right = 0
    left = count 
    for i in range(len(tab)-1, -1, -1):
        if i%2 == 1:
            left -= tab[i]
        else:
            score_from_right += remaining*left*tab[i]
    
    min_score = score_from_inside + min(score_from_left, score_from_right)
        
    return min_score


def GreedyForbiddenTriples(votes):
    matrix_coapp = matrix_coapproval(votes)
    v = matrix_coapp.ravel().argmax()
    v_x, v_y = np.unravel_index(v, matrix_coapp.shape)
    axis_out = [v_x, v_y]
    n, m = votes.shape
    for k in range(m-2):
        on_left = np.ones(m)*m*n
        on_right = np.ones(m)*m*n
        for i in range(m):
            if i not in axis_out:
                
                tested_axis = [i] + axis_out
                on_left[i] = sum([minForbiddenTriplesScore(tested_axis, vote) for vote in votes])
                tested_axis = axis_out + [i]
                on_right[i] = sum([minForbiddenTriplesScore(tested_axis, vote) for vote in votes])
                
        if np.min(on_left) < np.min(on_right):
            v = np.argmin(on_left)
            axis_out = [v] + axis_out
        else:
            v = np.argmin(on_right)
            axis_out = axis_out + [v]
    return axis_out

print("ForbiddenTriples Completion")
t1 = time.time()
res = GreedyForbiddenTriples(votes_spo)
axis.print_axis(res, parties)
t2 = time.time()
print("Time :", t2-t1)

ForbiddenTriples Completion
LR < HOR < RE < Dem < SOC < LFI < Écolo < GDR < LIOT < RN
Time : 0.8751695156097412


In [17]:
from PQTree import PQTree


def PQTreeAxis(votes, m):
    matrix  = axis.compute_weighted_matrix(votes, m)
    
    t = PQTree(list(range(m)))
    
    for row in matrix:
        vote = row[:-2]
        t.SafeReduce([i for i in range(m) if vote[i] == 1])

    return (list(t.Frontier()))
t1 = time.time()
print("PQTree Axis")
axis_out = PQTreeAxis(votes_spo, len(parties))
axis.print_axis(axis_out, parties)
t2 = time.time()
print("Time :", t2-t1)

PQTree Axis
LIOT < GDR < Écolo < LFI < SOC < RN < LR < HOR < RE < Dem
Time : 0.02781224250793457


In [44]:
print(np.std([3]*10+[2]+[-1]))
print(np.std([3]*10+[1]+[-1]))


1.3844373104863459
1.1902380714238083
